# Motion magnification
Motion magnification is a processing technique which amplifies small transverse quasi-periodic motions of contrasted features in image sequences: it acts as a **microscope for videos**.

Consider the humble microscope: through amplifying the image of small objects, microscopes have allowed us to view the world of tiny things the human eye cannot see. They led to some of the greatest scientific breakthroughs in history - for example cellular biology, forensic science, the existence of bacteria + viruses etc. Can we do the same thing but to small spatial scales changing over time in a video?

In [4]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML # To display video
from base64 import b64encode
from google.colab import drive
drive.mount('/content/gdrive')

# Show video
mp4 = open('/content/gdrive/MyDrive/sergey.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


This is [Dr Sergey Anfinogentov](https://orcid.org/0000-0002-1107-7420). He has provided everyone with a [code on Github](https://github.com/Sergey-Anfinogentov/motion_magnification) which can motion magnify any video. I aim to help you run motion magnification, and explain how it works!

You can see in the video above, that whilst the video on the left looks quite static, there are a whole range of motions which are much more obvious on the right. **These motions were always there**.

This processing tool has been used by me and others for use in solar physics, notably Sihui Zhong, Sudip Mandal, Yuhang Gao & Valery Nakariakov, all of whom are  here today! Hi! By magnifying the transverse motions of coronal structures seen in EUV, we have been able to study in great detail the regime of *decayless kink oscillations*.

These MHD waves are everywhere in the corona, so it is highly likely you could motion magnify any image sequence of the Sun where coronal structures are resolved (e.g. SDO/AIA 17.1 nm) and find some of these transverse waves. To prove me right/wrong, try going through the [mm_solardata]() notebook and give it a go!   

# Working example
Let us do some motion magnification as a "black box" process. Since we can use any video, I have picked one of my *gorgeous* dog Daphne.

In [7]:
# Input video path
daphne_path = "/content/gdrive/MyDrive/daphne_garden.MOV"
#"/content/gdrive/MyDrive/daphne_sleeping_compressed.mp4"

# Show video
mp4 = open(daphne_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
import cv2

input_video_filename = compressed_path

videoReader = cv2.VideoCapture(input_video_filename)  # create video object
nFrames = int(videoReader.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(videoReader.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(videoReader.get(cv2.CAP_PROP_FRAME_HEIGHT))

print("Total Number of Frames = ", nFrames)
print("Each frame has dimensions = ",frameHeight,"x", frameWidth) # Which way around?

# Note we specify float32 since necessary for cv2.color_bgr2gray
img_array_rgb = np.zeros((nFrames, frameHeight,frameWidth, 3), np.dtype("float32"))
img_array = np.zeros((nFrames, frameHeight,frameWidth), np.dtype("float32"))

nf = 0
ret = True

while (nf < nFrames  and ret):  # Could do for loop as well
#		print("frame: " + str(nf))
    ret, img_array_rgb[nf] = videoReader.read()
    img_array[nf] = cv2.cvtColor(img_array_rgb[nf], cv2.COLOR_RGB2GRAY)
    nf += 1
#		if ret == False or img is None:
#				print("done")
#				break

#videoWriter = make_video_writer(videoReader, output_video_filename)

videoReader.release()

# History
There have been multiple routines developed for motion magnification since around 2005. The early attempts focussed on directly estimating the velocity field between frames, in the so-called *Lagrangian* approach.  